> # FRA532 MOBILE ROBOT
> # LAB1: Local Planner
> **Sirawich Lousomboon 63340500066** <br>
> **Athimet Aiewcharoen 63340500068** <br>

> ## Launch procedure

- ros2 launch zhbbot_control zhbbot_velo_drive_con.launch.py 
- ros2 launch zhbbot_control zhbbot_rviz_nav2.launch.py 
- ros2 service call /zhbbot_service/user_setgoal zhbbot_interfaces/srv/ZhbbotUserSetgoal "{x: 0.0, y: 5.0, theta: 0.0}"


You can Edit local planer mode by modify zhbbot_control_params.yaml in zhbbot_control/config


<center> <img src="Pic/selectlocalplaner.png"/> </center>

> ## Algorithm 1: Pure-Pursuit with VFF Avoidance

### System Architecture


<center> <img src="Pic/PurePursuitVFF.png"/> </center>

**System Architecture**

The system consist of 5 Nodes which are:
1. zhbbot_handler
2. zhbbot_local_planner_vff_avoidance
3. zhbbot_inverse_kinematic
4. zhbbot_forward_kinematic
5. Gazebo Simulation


### Node Algorithm

**zhbbot_handler**

This node acts as a handler of a system which will receive a goal position (x,y) from a user. The user can input or config the goal position by calling the service through the topic "/zhbbot_service/user_setgoal" through the 2 arguments given which are x and y coordinate. After that, this service will response to ensure that this node receive the coordinate correctly.

After received the goal position from a user. This node will compute that robot path by using ComputePathtoPose from NAV2 Action Server and after that it will sent the robot path to Local Planner Node through the service called "/zhbbot_service/send_path"

**zhbbot_local_planner_vff_avoidance**

This node is a Local Planner Node that will receive a robot path from handler. After that, this node will create a local planner for a robot path by breaking down the robot path into many robot sub-paths and calculate the robot linear velocity (x-coordinate) and angular velocity (z-coordinate) for each sub-path from 2 algorithm which are:
1. Pure-Pursuit Algorithm
2. Virtual Force Field Obstacle Avoidance (VFF Avoidance)

Pure-Pursuit Algorithm
The Pure-Pursuit Algorithm is a path tracking technique used to navigate along a robot sub-paths. This algorithm starts by selecting a point within the search radius around the robot and choose it as a pursuit point. After that this algorithm will sent the coordinate of the pursuit point to the VFF Avoidance Algorithm to calculate the linear velocity and angular velocity for a robot to move to a pursuit point. When the robot moves until the next point in the robot sub-paths is withing the search radius, the algorithm will change the pursuit point to the next point. This algorithm loops until it reach the goal position.

Virtual Force Field Obstacle Avoidance
The VFF Avoidance Algorithm is a vector-base local planner which helps the robot to avoid the obstacle. The algorithm receives a pursuit point from a Pure-Pursuit Algorithm and robot position from robot odom to calculate the vector that leads to a pursuit point called "Pursuit Vector" (Attractive Vector). After that, it will check if there is an obstacle near the robot. If there is one, the algorithm will finds the nearest obstacle by getting the scan data from the Lidar Sensor from the topic "/scan" and calculate the vector that leads to the opposite side of an obstacle called "Avoidance Vector" (Repulsive Vector). Then the algorithm will combine these 2 vectors to create a "Result Vector" which is a vector that leads a robot to a pursuit point and avoid the obstacle at the same time. Finally, the algorithm will calculate the Linear Velocity and Angular Velocity and publish them along the topic "/diff_drive_zhbbot"

**zhbbot_inverse_kinematic**

This node is a kinematic node that will compute wheels velocity from a Linear Velocity and Angular Velocity sent by local planner node. The node calculated by using the Inverse Kinematic Method for a velocity controller to control the robot in Gazebo System Simulation. After the algorithm done calculating, it will publish the wheels velocity to a velocity command of a velocity controller through the topic "velocity_cont/comm"

**zhbbot_forward_kinematic**

This node is a kinematic node that will receive a Wheels Velocity from Gazebo Simulation through the topic "/joint_states" to compute wheels odometry. It starts by calculating Linear Velocity and Angular Velocity of a robot in Gazebo Simulation. And after that, it will compute the wheels odometry and sent it to the EKF Localization through the topic "/zhbbot_wheel/odom".

The Extended Kalman Filter Localization will receive wheels odometry and calculate the Robot Position and Orientation (Robot Odometry) by using the EKF Localization method. Then it will publish robot odometry through the topic "/odometry/local"

> ## Algorithm 2: Dynamic Window Approach

### System Architecture

<center> <img src="Pic/DynamicWindowApproach.png"/> </center>

**System Architecture**

The system consist of 5 Nodes which are:
1. zhbbot_handler
2. zhbbot_local_planner_dwa
3. zhbbot_inverse_kinematic
4. zhbbot_forward_kinematic
5. Gazebo Simulation

All the nodes acts the same as Pure-Pursuit VFF Algorithm except zhbbot_local_planner_dwa

### Node Algorithm

**zhbbot_local_planner_dwa**

This node is a Local Planner Node that will receive a robot path from handler. After that, this node will create a local planner for a robot path by breaking down the robot path into many robot sub-paths and calculate the robot linear velocity (x-coordinate) and angular velocity (z-coordinate) for each sub-path by using Dynamic Window Approach.

The Dynamic Window Approach will select the pursuit point (as same as Pure-Pursuit Algorithm) and then it will generates many future robot paths. After that it will select the best path by selecting the lowest cost path after done calculating the cost of travelling through each path. The cost of each path is determined by 1. the distance left between the future position and the goal 2. the distance apart from the obstacle (if the path contain or near the obstacle). After done selecting, the algorithm will calculate Linear Velocity and Angular Velocity for that path and publish through the topic "/diff_drive_zhbbot". This loop goes on until the robot reaches the goal position.

<center> <img src="Pic/DWA_psuedo1.png"/> </center>
<center> Dynamic Window Approach Pseudocode #1 </center>

<center> <img src="Pic/DWA_psuedo2.png"/> </center>
<center> Dynamic Window Approach Pseudocode #2 </center>

<center> <img src="Pic/DWA_psuedo3.png"/> </center>
<center> Dynamic Window Approach Pseudocode #3 </center>

> ## Experimentation

### Introduction

Pure-Pursuit with VFF Avoidance and Dynamic Window Approach, Both are the algorithms used to create a local planner for a Mobile Robot. To test the efficacy of the algorithm, we implimented both of the algorithms with a robot running in simulation using Gazebo simulation and launch Rviz2 for monitorize.

### Method

The experiment will break down into 2 parts
1. Square Testing: To test the accuracy of the algorithm. We will control the robot to follow the square path by using each of the algorithm. After it reached the goal position, we will collect the position data from gazebo and compare it with the command position.
2. Obstacle Avoidance Testing: To test whether the algorithm can avoid the obstacle or not, we will command the robot to follow the path which has an obstacle and see if the robot can avoid the obstacle and still manage to continue the path until it reach the goal position.

### Result

**Square Testing**

After running test for 5 rounds per algorithm. The Pure-Pursuit with VFF Avoidance Algorithm can navigate through the course correctly with the estimated error around 0.015 meter. The Dynamic Window Approach Algorithm can navigate through the course correctly with the estimated error around 0.034 meter.

<center> <img src="Pic/PPS_table.png"/> </center>
<center> Pure-Pursuit with VFF Avoidance Result Table #1 </center>

<center> <img src="Pic/DWA_table.png"/> </center>
<center> Dynamic Window Approach Result Table #1 </center>

<center> <img src="Pic/PPS_square1.png"/> </center>
<center> Pure-Pursuit with VFF Avoidance testing with square path #1 </center>

<center> <img src="Pic/DWA_square1.png"/> </center>
<center> Dynamic Window Approach testing with square path #1 </center>

**Obstacle Avoidance Testing**

After running the test for 10 times, both of the algorithm managed to follow the path until the robot reached the goal point and avoided colliding with the obstacle set on the map. The different between both algorithm is that the Dynamic Window Approach algorithm used more time to avoid the obstacle and continue the path until it reach the goal point because this method use more time to calculate the cost of each path (More Path = More Run Time).

<center> <img src="Pic/PPStest1_1.png"/> </center>
<center> Pure-Pursuit with VFF Avoidance testing with obstacle #1 </center>

<center> <img src="Pic/PPStest1_2.png"/> </center>
<center> Pure-Pursuit with VFF Avoidance testing with obstacle #1 </center>

<center> <img src="Pic/PPStest2_1.png"/> </center>
<center> Pure-Pursuit with VFF Avoidance testing with obstacle #2 </center>

<center> <img src="Pic/PPStest2_1.png"/> </center>
<center> Pure-Pursuit with VFF Avoidance testing with obstacle #2 </center>

<center> <img src="Pic/DWAtest1_1.png"/> </center>
<center> Dynamic Window Approach testing with obstacle #1 </center>

<center> <img src="Pic/DWAtest1_2.png"/> </center>
<center> Dynamic Window Approach testing with obstacle #1 </center>

<center> <img src="Pic/DWAtest2_1.png"/> </center>
<center> Dynamic Window Approach testing with obstacle #2 </center>

<center> <img src="Pic/DWAtest2_2.png"/> </center>
<center> Dynamic Window Approach testing with obstacle #2 </center>

### Conclusion

From the test result, the Pure-Pursuit with VFF Avoidance Algorithm manage to navigates through the course better than the Dynamic Window Approach Algorithm with better run time and position error. In addition, both algorithms did well in the obstacle avoidance test but Dynamic Window Approach Algorithm used more time to avoid the obstacle and continue the path until it reach the goal point than Pure-Pursuit with VFF Avoidance Algorithm.